In [16]:
# coding: utf-8

import numpy as np
import pandas as pd
import torch
import torchvision
from torchsummary import summary
import sys
sys.path.append("../")
from utils.tcn import TCN

import matplotlib.pyplot as plt
import matplotlib as mpl

#matplotlib inline
plt.rcParams['figure.figsize'] = 16,6
plt.rcParams['xtick.color'] = 'w'  
plt.rcParams['ytick.color'] = 'w'  
mpl.style.use('ggplot')
font1 = {'family' : 'Times New Roman' , 'weight' : 'normal','size': 23}
font2 = {'family' : 'Times New Roman' , 'weight' : 'normal','size': 18}

ImportError: cannot import name 'TCN'

In [4]:
#主要的超参数
"""
parser = argparse.ArgumentParser(description='1dcnn+blstm+fc')
parser.add_argument('--seed', default=0, type=int, help='random seed')
parser.add_argument('--sequence_length', default=110, type=int, help='sequence_length')
parser.add_argument('--lstm1', default=300, type=int, help='lstm1')
parser.add_argument('--lstm2', default=300, type=int, help='lstm2')
parser.add_argument('--dense1', default=50, type=int, help='dense1')
parser.add_argument('--dropout1', default=0.4, type=float, help='dropout1')
parser.add_argument('--dropout2', default=0.4, type=float, help='dropout2')
parser.add_argument('--epochs', default=1, type=int, help='epochs')
parser.add_argument('--batch_size', default=100, type=int, help='batch_size')
parser.add_argument('--lr', default=2e-2, type=float, help='learning rate')
parser.add_argument('--weight_decay', default=1e-4, type=float, help='weight_decay')
parser.add_argument('--test', default='0', type=str, help='input model file name, then it will be tested')
parser.add_argument('--filters1', default=16, type=int, help='filters1')
parser.add_argument('--filters2', default=32, type=int, help='filters2')

args = parser.parse_args()
print(args)
"""
args_seed = 0
args_sequence_length = 110
args_lstm1 = 300
args_lstm2 = 300
args_dense1 = 50
args_dropout1 = 0.4
args_dropout2 = 0.4
args_epochs = 10
args_batch_size = 100
args_lr = 2e-2
args_weight_decay = 1e-4
args_test = '0'
args_filters1 = 16
args_filters2 = 32

# seed
np.random.seed(args_seed)
torch.manual_seed(args_seed)
torch.cuda.manual_seed_all(0)

# ## 数据导入和处理
train_df_path = r'../data/processed_data/train_df.csv'
test_df_path = r'../data/processed_data/test_df.csv'
data_master_thesis_path = r'../../data/data_master_thesis'

train_df = pd.read_csv(train_df_path,index_col=0) #第一列作为index
test_df = pd.read_csv(test_df_path,index_col=0)

#print("train_df shape: {}".format(train_df.shape))
#print("test_df shape: {}".format(test_df.shape))


# ## 定义X_train, y_train, X_test, y_train


X_train = train_df.iloc[:,5:26]
y_train = train_df.iloc[:,-2]

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = test_df.iloc[:,5:26]
y_test = test_df.iloc[:,-1]#注意，test_df中RUL列是最后一列，
                           #但是，测试集的RUL不是test_df的RUL对应的列

X_test = np.array(X_test)
y_test = np.array(y_test)

#print("X_train.shape: {}, y_train.shape: {}, X_test.shape: {}, y_test.shape: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

In [5]:
# ## 采用时间窗分割的方式改变数据的维度


# 将数据格式变为(样本循环次数, 时间窗大小：30, 特征数)
def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(
            range(0, num_elements - seq_length), range(seq_length,
                                                       num_elements)):
        yield data_array[start:stop, :]


# 选择特征列
sensor_cols = ['s' + str(i) for i in range(1, 22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

# seq_array为用上函数生成的数组，其形状为(15631, 30, 25)
seq_gen = (list(
    gen_sequence(train_df[train_df['id'] == id], args_sequence_length,
                 sequence_cols)) for id in train_df['id'].unique())
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)


# 对应数据格式生成标签
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

# 标签的形状为(15631, 1)
label_gen = [
    gen_labels(train_df[train_df['id'] == id], args_sequence_length, ['RUL'])
    for id in train_df['id'].unique()
]
label_array = np.concatenate(label_gen).astype(np.float32)


# 生成test数据的最后一个序列，形状为(93, 50, 25)，不足100是因为有些测试集小于50
seq_array_test_last = [
    test_df[test_df['id'] == id][sequence_cols].values[-args_sequence_length:]
    for id in test_df['id'].unique()
    if len(test_df[test_df['id'] == id]) >= args_sequence_length
]
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

# 对应生成test的label，形状为(93, 1)
y_mask = [
    len(test_df[test_df['id'] == id]) >= args_sequence_length
    for id in test_df['id'].unique()
]
label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(
    label_array_test_last.shape[0], 1).astype(np.float32)

X_train = seq_array
y_train = label_array
X_test = seq_array_test_last
y_test = label_array_test_last

print("X_train.shape: {}, y_train.shape: {}, X_test.shape: {}, y_test.shape: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

# 现在X_train, y_train, X_test, y_test已经准备好了
def get_rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

X_train.shape: (9631, 110, 25), y_train.shape: (9631, 1), X_test.shape: (67, 110, 25), y_test.shape: (67, 1)


In [13]:
model = TemporalConvNet(num_inputs=[9631, 110, 25], num_channels=[25, 12, 12])
summary(model, X_train, batch_size=args_batch_size)

TypeError: unsupported operand type(s) for %: 'list' and 'int'